In [1]:
# Run the date count jobs tasks
from jobs.solana import SparkJob

In [2]:
job = SparkJob()
job.solona_union_dataframes()

preprocessing completed.
d1: counts: 5952
d2: counts: 5371
d3: counts: 5946
d1: counts: 5952
d2: counts: 5371
d3: counts: 5946
d123: counts: 17269
+----+----+---+-----+
|open|high|low|close|
+----+----+---+-----+
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
|   1|   1|  1|    1|
+----+----+---+-----+
only showing top 10 rows

